In [1]:
!pip3 install requests

In [2]:
import requests
import pandas as pd
import re
from datetime import datetime
# import warnings
# warnings.filterwarnings('ignore')

API_KEY = "d70f5128bf847e1e"
count = 10
keyword = '心斎橋駅'

URL = 'http://webservice.recruit.co.jp/hotpepper/gourmet/v1/'
body = {
    'key':API_KEY,
    'keyword':keyword,
    'range':2,
    'format':'json',
    'count':count
}

#API情報のすべて
response = requests.get(URL,body).json()
#ショップ数
# shop_count = response['results']['results_available']
# print(shop_count)
#shop情報すべて
df_shop = pd.DataFrame(response['results']['shop'])
# df_shop.info()

element = ["id","name","capacity","open","budget"]
# df_shop[element]

for z in range(0,10):
    t = df_shop['budget'][z]['average'].replace(",","")
    print(re.findall(r'\d+', t))


shop_info_list = []
for i in range(0,count - 1):
    shop_info = {}
    #総席数：shop_info_list
#     print(shop[i]['capacity'])
    shop_info['総席数'] = shop[i]['capacity']
    
    #営業時間：open}
#     shop_info['open'] = shop[i]['open']
    s = shop[i]['open']
    match = re.findall(r'\d+', s)
    #00:00の場合24:00にする
    if match[2] == "0":
        match[2] = "00"
        
    for a in range(0,4):
        if len(match[a]) == 1:
            match[a] = f"0{match[a]}"
    shop_info['オープン'] = f"{match[0]}:{match[1]}"
    shop_info['クローズ'] = f"{match[2]}:{match[3]}"

    #単価(平均予算ver)：budget_average
    t = df_shop['budget'][i]['average'].replace(",","")
    shop_info['単価'] = re.findall(r'\d+', t)[0]
#     pre_budget_average = df_shop['budget'][i]['average'].split('円')[0].replace(",","")
#     shop_info['単価'] = re.findall(r'\d+',pre_budget_average)[0]
    
    shop_info_list.append(shop_info)
df_shop_alt = pd.DataFrame(shop_info_list)
df_shop_alt

['3000', '4000']
['3000']
['3800']
['1000']
['3000', '4000']
['3000', '4000']
['2000']
['3000', '800']
['800', '2000', '5000', '7000']
['2000', '2000']


NameError: name 'shop' is not defined

In [3]:
df_shop_alt['営業時間'] = (pd.to_datetime(df_shop_alt['クローズ']) - pd.to_datetime(df_shop_alt['オープン']))/ timedelta(hours=1)

NameError: name 'df_shop_alt' is not defined

In [718]:
#24時間表記へ書き換え
n_24 = 0
for add24 in df_shop_alt['営業時間']:
    if add24 <= 0: 
        df_shop_alt['営業時間'][n_24] = df_shop_alt['営業時間'][n_24] + 24
    n_24 += 1
#来店人数 = 回転率 * 席数
df_shop_alt['回転率'] = 3
df_shop_alt['来店人数'] =  df_shop_alt['総席数'] * df_shop_alt['回転率']

##売り上げ(単価 * 来店人数
df_shop_alt['営業日数　'] = 23
df_shop_alt['売上'] = df_shop_alt['単価'].astype('int') * df_shop_alt['来店人数'] * df_shop_alt['営業日数　']

#人件費計算
df_shop_alt['時給'] = 1000
df_shop_alt['従業員数'] = 10

df_shop_alt['人件費'] = df_shop_alt['時給'] * df_shop_alt['従業員数'] * df_shop_alt['営業時間']

In [719]:
##経費(家賃 + 人件費 + 水道光熱費 + 消耗品) + (原価)
df_shop_alt['家賃'] = 200000
df_shop_alt['その他'] = 5

df_shop_alt['経費'] = (Rent + df_shop_alt['人件費'] + df_shop_alt['家賃'] + df_shop_alt['売上']*0.25).astype('int') +df_shop_alt['その他']

In [720]:
##利益
df_shop_alt['利益'] = df_shop_alt['売上'] - df_shop_alt['経費']

In [721]:
#識別
df_shop_alt["ID"] = df_shop["id"] 
df_shop_alt["掲載店名"] = df_shop["name"]

In [722]:
re_columns = [ 'ID','掲載店名','利益','売上', '単価','総席数','回転率','来店人数',
              '経費','人件費', '従業員数','時給', '営業時間','オープン', 'クローズ' , '家賃','その他'
       ]
df_shop_alt[re_columns]

,ID,掲載店名,利益,売上,単価,総席数,回転率,来店人数,経費,人件費,従業員数,時給,営業時間,オープン,クローズ,家賃,その他
0,J001192324,かつら亭 別館 心斎橋,4017495,6210000,3000,30,3,90,2192505,240000.0,10,1000,24.0,00:00,00:00,200000,5
1,J000012674,かつら亭 心斎橋店,7242495,10350000,3000,50,3,150,3107505,120000.0,10,1000,12.0,11:00,23:00,200000,5
2,J000020452,和食居酒屋 楽遊 心斎橋駅近店,5051195,7341600,3800,28,3,84,2290405,55000.0,10,1000,5.5,17:30,23:00,200000,5
3,J003497820,JAMES CAFE ジェームスカフェ 心斎橋パルコ店,4654995,6900000,1000,100,3,300,2245005,120000.0,10,1000,12.0,11:00,23:00,200000,5
4,J001223516,中華 広東料理 昌和 心斎橋店,3836995,5796000,3000,28,3,84,1959005,110000.0,10,1000,11.0,12:00,23:00,200000,5
5,J003324243,焼肉 肉王,4848495,7038000,3000,34,3,102,2189505,30000.0,10,1000,3.0,11:30,14:30,200000,5
6,J001282767,すし共友丸 南船場店,5629995,8280000,3000,40,3,120,2650005,180000.0,10,1000,18.0,18:00,12:00,200000,5
7,J001262242,RED SNAPPER レッドスナッパー,4941995,7176000,2000,52,3,156,2234005,40000.0,10,1000,4.0,11:00,15:00,200000,5
8,J003297406,焼肉 大,-76005,552000,800,10,3,30,628005,90000.0,10,1000,9.0,18:00,03:00,200000,5


In [701]:
df_shop_alt.astype("str").describe()
# df_shop_alt['利益'].value_counts()

,総席数,オープン,クローズ,単価,営業時間,回転率,来店人数,営業日数,売上,時給,従業員数,人件費,家賃,その他,経費,利益,ID,掲載店名
count,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
unique,8,6,6,5,8,1,8,1,9,1,1,8,1,1,9,9,9,9
top,28,11:00,23:00,3000,12.0,3,84,23,6210000,1000,10,120000.0,200000,100000,728000,4842000,J003297406,RED SNAPPER レッドスナッパー
freq,2,3,4,5,2,9,2,9,1,9,9,2,9,9,1,1,1,1


In [506]:
###何がしたいか？（ゴール
#1:店の売り上げ（利益）を知りたい
#2:赤字にならないための単価設定をしたい（来店人数固定ver
#3:赤字にならないための来店人数を知りたい（単価固定ver


###各パラメータを用意する
##固定---------------
#総席数：shop_capacity   
#家賃_Rent
#人件費_Labor
#水道光熱費_Utility
#消耗品_Consumption
#営業時間：open
#時給_HourlyPay

##変動---------------
#単価(平均予算ver)：budget_average
#従業員数_Labor_number

##調整---------------
#来店人数_CustomerNumber
#原価(25～30%程度)_Material
#滞在時間_StayTime

###目的変数-----------
##売り上げ
#単価 * 来店人数

##経費
#(家賃 + 人件費 + 水道光熱費 + 消耗品) + (原価)

##利益
#売り上げ - 経費


###補助計算----------------
##回転率
#来店人数 ÷ 席数

##人件費
#人数 * 時給 * 営業時間




